In [1]:
from dotenv import load_dotenv
import openai
import os
from openai import AzureOpenAI
from IPython.display import display, HTML, JSON, Markdown, Image
import base64 

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AISTUDIO_AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_GPT4o_DEPLOYMENT=os.getenv("AI_STUDIO_AZURE_OPENAI_GPT4o_DEPLOYMENT")
AZURE_OPENAI_API_VERSION="2024-02-01"
AZURE_OPENAI_KEY=os.getenv("AISTUDIO_AZURE_OPENAI_KEY")

client = openai.AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION
)

In [2]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [12]:
# Defining the functions to be called by the LLM
def query_knowledge_base(question):
     """Getting information from the knowledge base"""
     return f"Getting information from the knowledge base on the following question:  {question}"

def query_database(queryParams):
    """Getting DB data"""
    # you may use the current user's context to query the database
    return f"Querying DB with: {queryParams}"


In [14]:
# Defining the available functions and their metadata to send to GPT-4o
tools = [
    {
        "type": "function",
        "function": {
            "name": "query_kbase",
            "description": "Query the knowledge base for general information about shipping methods and prices",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask the knowledge base.",
                    }
                },
                "required": ["question"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "query_db",
            "description": "Query the database for a parcel location, tracking data, parcel data and user data",
            "parameters": {
                "type": "object",
                "properties": {
                    "queryParams": {
                        "type": "string",
                        "description": "The parameters to query the database in JSON format of the form {\"table\": \"table_name\", \"columns\": [\"column1\", \"column2\"], \"conditions\": {\"column1\": \"value1\", \"column2\": \"value2\"}}",
                    }
                },
                "required": ["queryParams"],
            },
        },
    },
]

available_functions = {
    "query_kbase": query_knowledge_base,
    "query_db": query_database
}

In [15]:
import json

def call_OpenAI(question):
    messages = [
        {"role":"system","content":"""You are a customer service assistant for a delivery service, 
         equipped to analyze user questions.
         Based on the user question you can query a knowledge base or a database to get more information.
         You must always use tools!"""
        },
        {"role":"user","content":question}
    ]

    # Step 1: send the prompt and available functions to GPT
    response = client.chat.completions.create(
        model=AZURE_OPENAI_GPT4o_DEPLOYMENT,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    response_message = response.choices[0].message

    # Step 2: check if GPT wanted to call a function
    if response_message.tool_calls:
        print("Recommended Function call:")
        print(response_message.tool_calls[0])
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        function_name = response_message.tool_calls[0].function.name

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(response_message.tool_calls[0].function.arguments)
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        # call the function
        function_response = function_to_call(**function_args)
        print("Output of function call:")
        print(function_response)
        print()


In [17]:
question = "How much does it cost to ship a parcel to the UK?"
call_OpenAI(question)

Recommended Function call:
ChatCompletionMessageToolCall(id='call_vg3prHOOp0JV4Ch830s7FJrK', function=Function(arguments='{"question":"How much does it cost to ship a parcel to the UK?"}', name='query_kbase'), type='function')

Output of function call:
Getting information from the knowledge base on the following question:  How much does it cost to ship a parcel to the UK?



In [18]:
question = "Where is parcel number 123?"
call_OpenAI(question)

Recommended Function call:
ChatCompletionMessageToolCall(id='call_3JaYAjChXeP4DFKHtYC4051d', function=Function(arguments='{"queryParams":"{\\"table\\":\\"parcels\\",\\"columns\\":[\\"location\\"],\\"conditions\\":{\\"parcel_number\\":\\"123\\"}}"}', name='query_db'), type='function')

Output of function call:
Querying DB with: {"table":"parcels","columns":["location"],"conditions":{"parcel_number":"123"}}

